# 🐅 Парсер
Сначала проверяются главные («стартовые») страницы, описанные в файле `pages.json`. Далее найденные на них ссылки на внутренние страницы мероприятий. Результаты записываются в файл `events.json`, который далее используется для генерации карточек мероприятий в HTML. Результаты публикуются на [matyushkin.github.io/events](https://matyushkin.github.io/events/).

In [12]:
import json

# custom packages
import files
import soups
import urls
import langs
import handlers

# Узнаем, какие из стартовых страниц, перечисленных в files.pages, размечены.
# Такие страницы содержат все поля, перечисленные в files.fields.
# Выводим список с помощью специальной функции pages_checked()

pages_checked = files.pages_checked()

# Для обработки полей, описывающих события, логична следующая последовательность:
# - cчитываемые поля со стартовой страницы (обычно это заголовк, дата, статус)
# - поля, расположенные на странице мероприятия (темы, докладчики, время начала)
# - получаемые из анализа предобработанных полей (теги, призы)


class StartPage:
    """Парсинг начальных страниц, описанных в pages.json"""
    def __init__(self, start_url):
        self.data, self.events = {}, {}
        self.data['start_url'] = start_url
        self.fields = files.fields_order(start_url)
        for field in self.fields['start']:
            self.data[field] = files.get_content(self.data, field)
        for i, event_url in enumerate(self.data['event_url']):
            self.events[event_url] = {}
            for field in set(self.fields['start']) - {'event_url'}:
                self.events[event_url][field] = self.data[field][i]
    

    def __repr__(self):
        return self.actual_events
            
                
    @property
    def actual_events(self):
        """События, актуальные на текущий день"""
        events = self.events.copy()
        for event_url in self.events:
            if events[event_url]['date'] < handlers.current_date.isoformat():
                events.pop(event_url)
        return events


class EventPage:
    """Парсинг страниц событий"""
    def __init__(self, event_url, StartPage):
        self.data = StartPage.events[event_url]
        self.data['start_url'] = StartPage.events['start_url']
        self.data['event_url'] = event_url
        self.event_fields = page.event_fields
        for field in StartPage.fields['event']:
            self.data[field] = files.get_content(self.data, field)


for name in pages_checked:
    page = StartPage(name)
    actual = page.actual_events
    for event_url in actual:
        event = Event(event_url, page)
        files.events[event_url] = event.data

{'start': ['title', 'event_url', 'date'], 'event': ['description', 'organizers', 'location', 'time', 'themes', 'price', 'speakers', 'reg_url', 'online_status', 'speakers_companies'], 'analysis': ['honor_description', 'language', 'prize', 'tags', 'event_type']}


NameError: name 'url' is not defined

In [2]:
with open('files/events.json', 'w', encoding='utf-8') as events_file:
    json.dump(files.events, events_file, ensure_ascii=False)

In [3]:
#! Отсюда можно забрать обработчикитегов

# all_fields = set(fields.keys())
# required   = {key for key in fields if fields[key]['required']}
# optional   = {key for key in fields if not fields[key]['required']}

#     def tags(self):
#         tags = {}
#         title = self.data['title']
#         event_url = self.data['event_url']
#         themes = self.data['themes'].copy()
#         themes.append(title)
#         themes.append(event_url)
#         text = ''.join(themes)
#         for key in tags:
#             for tag in tags[key]:
#                 if tag.lower() in text.lower():
#                     self.data[tags].add(tag)
#         return tags

In [13]:
d = {'a':1, 'b':2}

In [16]:
print(d.get('c'))

None


# Собираем HTML-страницу и деплоим проект 💃

In [4]:
from bs4 import BeautifulSoup, NavigableString
import pandas as pd

soup = BeautifulSoup('<article></article')
    
df = pd.DataFrame.from_dict(files.events, orient='index')
df = df.sort_values(by=['date'])
df = df[df['date'] >= handlers.current_date.isoformat()]   #! добавить проверку на время


def add_tag(child, parent, text='', attrs={}):
    tag = soup.new_tag(child, attrs=attrs)
    tag.string = text
    parent.append(tag)
    return eval(f'parent.{child}')


for event_url in df.index:
    data = df.loc[event_url]
    attrs = {'data-date': data.date, 'data-time': data.time, 'data-organizers': data['organizers']}
    section = soup.new_tag('section', attrs)
    hr = soup.new_tag('hr')
    section.append(hr)

    h2 = add_tag('h2', section, data.title)
    p = add_tag('p', section)
    datetime_str = langs.make_datetime_string(data.date, data.time)
    t = add_tag('time', p, datetime_str)
    p.append(NavigableString(", "))
    add_tag('a', p, text='ссылка на мероприятие', attrs={'href': event_url})
    
    if data.reg_url:
        p.append(NavigableString(", "))
        add_tag('a', p, text='ссылка для регистрации', attrs={'href': data.reg_url})

    if data.themes != ['']:
        themes_ul = add_tag('ul', section, attrs={'class': 'theme-block'})

        for theme in set(data.themes):
            theme_li = add_tag('li', themes_ul, theme, {'class': 'theme'})

    description = add_tag('p', section, data.description[0], attrs={'class':'description__text'})
    soup.article.append(section)

hr = soup.new_tag('hr')
soup.article.append(hr)

path_to_html_template = "../mgio/11ty/_includes/events.njk"
with open(path_to_html_template, 'w', encoding='utf-8') as html_template:
    html_template.write(str(soup.article))

In [5]:
!cd ../mgio/11ty/; npx eleventy --passthroughall --output=../../matyushkin.github.io
!cd ../matyushkin.github.io/; rm -rf 404
!cd ../matyushkin.github.io/; rm -rf README


# посмотрим, что получилось в браузере
import webbrowser
url = "../matyushkin.github.io/events/index.html"
webbrowser.open_new_tab(url)

Writing ../../matyushkin.github.io/404/index.html from ./404.html.
Writing ../../matyushkin.github.io/index.html from ./index.html.
Writing ../../matyushkin.github.io/README/index.html from ./README.md.
Writing ../../matyushkin.github.io/cv/index.html from ./cv/index.html.
Writing ../../matyushkin.github.io/donate/index.html from ./donate/index.html.
Writing ../../matyushkin.github.io/events/index.html from ./events/index.html.
Writing ../../matyushkin.github.io/links/index.html from ./links/index.html.
Writing ../../matyushkin.github.io/texts/index.html from ./texts/index.html.
Writing ../../matyushkin.github.io/spb/index.html from ./spb/index.html.
Writing ../../matyushkin.github.io/posts/index.html from ./posts/index.html.
Copied 15 files / Wrote 10 files in 0.19 seconds (19.0ms each, v0.11.0)


True

In [6]:
! cd ../matyushkin.github.io/; git add . ; git commit -m "Events page is updated: only actual events"; git push origin master

[master b9839b6] Events page is updated: only actual events
 1 file changed, 10 insertions(+), 10 deletions(-)
 rewrite events/index.html (83%)
Counting objects: 4, done.
Delta compression using up to 8 threads.
Compressing objects: 100% (3/3), done.
Writing objects: 100% (4/4), 489 bytes | 489.00 KiB/s, done.
Total 4 (delta 2), reused 0 (delta 0)
remote: Resolving deltas: 100% (2/2), completed with 2 local objects.
To github.com:matyushkin/matyushkin.github.io.git
   82310eb..b9839b6  master -> master
